In [1]:
from scapy.all import *

# request / reponce icmp

In [2]:
icmp_request = IP(dst='8.8.8.8') / ICMP(type=8) / "Hello, this is a test"
response = sr1(icmp_request, timeout=2)

if response:
    print('Respnse received')
    response.show()
else:
    print('No reponse from the server')

Begin emission:
Finished sending 1 packets.

Received 13 packets, got 1 answers, remaining 0 packets
Respnse received
###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x80
  len       = 49
  id        = 0
  flags     = 
  frag      = 0
  ttl       = 113
  proto     = icmp
  chksum    = 0x170a
  src       = 8.8.8.8
  dst       = 10.80.23.227
  \options   \
###[ ICMP ]### 
     type      = echo-reply
     code      = 0
     chksum    = 0x5c47
     id        = 0x0
     seq       = 0x0
     unused    = ''
###[ Raw ]### 
        load      = 'Hello, this is a test'



# Simulating icmp error

In [3]:
ip_header = IP(src="192.168.1.100", dst="8.8.8.8")
udp_segment = UDP(sport=1234, dport=80)
icmp_error = IP(src="8.8.8.8", dst="192.168.1.100") / ICMP(type=3, code=3) / ip_header / udp_segment 
"""
    In ICMP errors, the payload must contain:

    ✔ the original IP header
    ✔ the first 8 bytes of the transport header (UDP in this case)

    RFC 792 requires this.
"""

send(icmp_error)
print("ICMPO destination Unreacheable message sent")


Sent 1 packets.
ICMPO destination Unreacheable message sent


# Scanning a network

In [4]:
def scan_network(network):
    print(f'Scanning the network {network}')
    # generate ICMP Echo Request packets for each IP address on the network
    ans, unans = sr(IP(dst=network) / ICMP(type=8), timeout=2, verbose=0) # sr() for send packets for each ip in the network & receive , ans := all packets that received a reply (alive) unans := packets did not received a reply
 
    # process responses received
    print('Host responding to ICMP Echo Request:')
    for sent, received in ans:
        print(f'{received.src} is alive.')
    
    # Non-reponding hosts
    if unans:
        print('Host not reponding.')
        for sent in unans:
            print(sent.dst)

In [5]:
scan_network("10.80.1.0/30")

Scanning the network 10.80.1.0/30


Host responding to ICMP Echo Request:
10.80.1.1 is alive.
Host not reponding.
10.80.1.0
10.80.1.2
10.80.1.3


# Personalized ping

In [6]:
# SENDER side

def send_ping(target_ip):
    message = "Hello from sender!"
    icmp_request = IP(dst=target_ip) /ICMP(type=8) / message

    print(f'Sending ICMP Echo Resequest to {target_ip}')
    response = sr1(icmp_request, timeout=2)

    if response:
        print('Received reponse:')
        response.show()
    else:
        print('No response received')

In [7]:
# RECEIVER side

def handle_packet(packet):
    if (ICMP in packet and packet[ICMP].type==8):
        print("ICMP Echo Request received:")
        packet.show()

        reply = IP(src=packet[IP].dst, dst=packet[IP].src) / ICMP(type=0) / "Hello from the other side!"
        print("Sendind ICMP Echo Reply...")
        send(reply, verbose=0)
